In [56]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy

In [57]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection": "keep-alive",
    "Host": "you.ctrip.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}

In [58]:
url_1='https://you.ctrip.com'
#获取评论者信息
def get_reviewer(url):
    reviewer=[]
    if url == 'https://you.ctrip.com':
        return ['','','','']
    else:
        html = requests.get(url,timeout=20)
        soup = BeautifulSoup(html.content,'html.parser')
    
        #名称信息
        targets_1 = soup.find('span',class_="info-name")
    
        name= targets_1.text
        reviewer.append(name)
    
        #性别信息
        targets_2 = soup.find('span',class_="J_gender")
        gender = targets_2.find('i')['class'][0]
        reviewer.append(gender)
    
        #关注信息
        target_3 = soup.find('div',class_="info-side")
        k=1
        for string in target_3.stripped_strings:
            if k%2 !=0:
                reviewer.append(string)
            k+=1
        return(reviewer)

In [53]:
def get_data(poiID,n):
    data=list()
    for i in range(16,n+1):
        url_raw="http://you.ctrip.com/destinationsite/TTDSecond/SharedView/AsynCommentView?poiID="+str(poiID)+"&districtId=702&districtEName=Yangshuo&pagenow=%d&order=3.0&star=0.0&tourist=0.0&resourceId=22079&resourcetype=2"
        url=url_raw%(i)
        html=requests.get(url,headers=headers)
        html.encoding="utf-8"
        soup=BeautifulSoup(html.content)
        block=soup.find_all(class_="comment_single")
        for j in block:
            item=[] 
            
            #评论者信息
            url_Reviewer=j.find(itemprop="author")['href']
            Reviewer=get_reviewer(url_1+url_Reviewer)
            
            #评论文本
            text=j.find(class_="heightbox").text
            
            #星级
            style=j.find('span',style =re.compile('width'))
            
            #评论时间
            time = j.find(class_="time_line").text
            
            #有用数
            useful = j.find(class_="useful").text
            item=item+Reviewer
            item.append(time)
            item.append(style['style'])
            item.append(text)
            item.append(useful)
            data.append(item)
            
    review_data=pd.DataFrame(data,columns=['name','gender','关注','粉丝','time','rating','text','useful'])
    return(review_data)

In [54]:
df=get_data(10770546,16)

In [55]:
outputpath='F:/太仓现代农业园16.csv'
df.to_csv(outputpath,sep=',',index=True,header=True,encoding='utf_8_sig')